# Estimación de dirección de arribo de fuentes sonoras 

In [1]:
import numpy as np
import pandas as pd
import gspread # Extension para leer el excel desde Drive
from oauth2client.service_account import ServiceAccountCredentials # Extensión para autentificación con Drive
from tdoa_py import correlation, simulation_room

In [2]:
# Configurar credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)

# Abrir la hoja de cálculo
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=0#gid=0")
sheet = spreadsheet.sheet1

# Leer datos
data = sheet.get_all_records()
df_simulations = pd.DataFrame(data)

df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1) # Para pasar el recinto en una tupla luego
df_simulations.head() # Verificar el resultado

,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,src_az_deg,src_z,theta_1,theta_2,theta_3,avg_angle_deg,avg_tdoa,hemi_1,hemi_2,room_dim
0,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,5,0.1,...,45,1.2,44.4,44.4,224.4,44.4,0.0,44.4,224.4,"(100, 100, 100)"


In [3]:
# Iterar sobre el DataFrame con indexación correcta
for idx, sim in df_simulations.iterrows():
    src_az_deg, mic_signals = simulation_room.sim_room_Nmics(
        wav_path=f'audios/anechoic/{sim["audio"]}',
        out_dir=f'audios/output/{sim["sim_name"]}',
        fs=48000,
        room_dim=sim["room_dim"],
        rt60=sim["rt60"],
        snr_db=sim["snr_db"],
        n_mics=sim["n_mics"],
        mic_d=sim["mic_d"],
        mic_z=sim["mic_z"],
        mic_directivity=sim["mic_directivity"],
        src_dist=sim["src_dist"],
        src_az_deg=sim["src_az_deg"],
        src_z=sim["src_z"],
        save_audio=False
    )
    
    avg_angle_deg, avg_tdoa, hemi_avgs, tdoas, angles = correlation.estimate_doa(
        signal_input=mic_signals,
        mic_d=sim["mic_d"],
        method=sim["method"],
        fs=48000
    )

    # Guardar resultados en cada fila individual
    df_simulations.at[idx, 'avg_angle_deg'] = avg_angle_deg
    df_simulations.at[idx, 'avg_tdoa'] = avg_tdoa
    df_simulations.at[idx, 'hemi_1'] = hemi_avgs['H1']
    df_simulations.at[idx, 'hemi_2'] = hemi_avgs['H2']
    df_simulations.at[idx, 'theta_1'] = angles[0]
    df_simulations.at[idx, 'theta_2'] = angles[1]
    df_simulations.at[idx, 'theta_3'] = angles[2]

# Convertir el DataFrame a una lista de listas para Google Sheets
df_simulations["room_dim"] = df_simulations["room_dim"].apply(lambda x: ",".join(map(str, x)))
data = [df_simulations.columns.values.tolist()] + df_simulations.values.tolist()

# Escribir en la hoja de cálculo
sheet.update(data)
print("DataFrame guardado correctamente en Google Sheets.")

df_simulations



5
2
DataFrame guardado correctamente en Google Sheets.


,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,src_az_deg,src_z,theta_1,theta_2,theta_3,avg_angle_deg,avg_tdoa,hemi_1,hemi_2,room_dim
0,1,classic,audio_anecoico_corto.wav,100,100,100,3,70,5,0.1,...,45,1.2,44.390938,44.390938,224.390938,44.390938,0.000005,44.390938,225.818385,"100,100,100"
